<a href="https://colab.research.google.com/github/CodeCraftIA/imdb-dead-actors-scraper/blob/main/imdb_actor_deaths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
from logging import exception
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm
import time

def extract_lines(input_file_path, start_line, end_line, output_file_path):
    try:
        with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
            lines = input_file.readlines()
            # Adjusting start_line and end_line to be within the valid range
            start_line = max(1, min(start_line, len(lines)))
            end_line = max(start_line, min(end_line, len(lines)))

            # Writing the selected lines to the output file
            output_file.writelines(lines[start_line - 1:end_line])

        print(f"Lines {start_line} to {end_line} extracted and saved to {output_file_path}")

    except Exception as e:
        print(f"Error: {e}")


def scrape_data(url, name):
    # Define the custom headers
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)

    if not response.ok:
        print('Status Code:', response.status_code)
        raise Exception('Failed to fetch: ', url)

    soup = BeautifulSoup(response.text)

    # Initialize variables to store the extracted information
    born_date = ""
    born_location = ""
    death_date = ""
    death_cause = ""
    personal_det = soup.find('section', {'data-testid': 'PersonalDetails'})
    if personal_det:
      lis = personal_det.find_all('li', class_='ipc-metadata-list__item')
      # Iterate over each list item to find the relevant details
      for li in lis:
          tag = li.find('span', class_='ipc-metadata-list-item__label')
          if tag:
            tag = tag.text.strip()
          if tag == 'Born':
              # Extract the born date
              born_info = li.find_all('a')
              try:
                born_date = f"{born_info[0].text}, {born_info[1].text}"  # e.g., "December 9, 1929"
              except Exception as e:
                born_date =""
              try:
                # Extract the born location
                born_location = born_info[2].text  # This assumes the location is the third <a> tag
              except Exception as e:
                born_location =""

          elif tag == 'Died':
              # Extract the death date
              death_info = li.find_all('a')
              try:
                # Extract the born location
                death_date = f"{death_info[0].text}, {death_info[1].text}"  # e.g., "February 3, 1989"
              except Exception as e:
                death_date =""

              try:
                # Extract the cause of death
                death_cause_span = li.find('span', class_='ipc-metadata-list-item__list-content-item--subText')
                if death_cause_span:
                    death_cause = death_cause_span.text.strip()
                    death_cause = death_cause.replace('(', '').replace(')', '')
                break
              except Exception as e:
                death_cause =""

    #Return a Dictionary

    return{
        'Name': name,
        'IMDB URL': url,
        'Born Date': born_date,
        'Born Location': born_location,
        'Death Date': death_date,
        'Cause of Death': death_cause,
    }


def iterate_urls(file_path):
    # Read URLs and names from the file
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()

    # Scrape data for each URL
    results = []
    for line in tqdm(lines):
        try:
            url, name = line.split(' | ')  # Split the line into URL and name
            data = scrape_data(url, name)  # Assuming scrape_data only needs the URL
            results.append(data)
        except Exception as e:
            print(f"Error scraping data for URL: {url}\nError: {e}")
            # Write the error URL and name to errors.txt
            with open('errors_imdb.txt', 'a') as error_file:
                error_file.write(f"{url} | {name}\n")

    return results

In [ ]:
import pandas as pd
def write_excel(items, path):
    if len(items) == 0:
        return

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(items)

    # Write DataFrame to Excel
    with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
        df.to_excel(writer, index=False, sheet_name='Sheet1')

In [ ]:
extract_lines('imdb_mexico_all.txt', 1701, 3300, 'file.txt')

Lines 1701 to 3300 extracted and saved to file.txt


In [ ]:
# Example usage:
urls_file_path = 'file.txt'
output_csv_path = 'imdb_all3.xlsx'

# Get scraped data for each URL
scraped_data = iterate_urls(urls_file_path)

# Write the results to a CSV file
write_excel(scraped_data, output_csv_path)

100%|██████████| 1600/1600 [36:39<00:00,  1.37s/it]


In [ ]:
# Write the results to a CSV file
write_excel(scraped_data, output_csv_path)

Wikipedia

In [ ]:
from logging import exception
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

# Define the custom headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
}
offset = 1000
while offset < 9501:
  url = f"https://es.wikipedia.org/w/index.php?limit=500&offset={offset}&profile=default&search=actor+mexicano&title=Especial:Buscar&ns0=1&ns100=1&ns104=1"
  response = requests.get(url, headers=headers)
  if not response.ok:
      print('Status Code:', response.status_code)
      raise Exception('Failed to fetch')

  soup = BeautifulSoup(response.text)
  content = soup.find('ul', class_='mw-search-results')
  # Find all elements that match the search result headings
  results = content.select("div.mw-search-result-heading a")

  # Extract the href and text (name) from each element
  links_and_names = [("https://es.wikipedia.org/" + result['href'], result.text) for result in results]

  with open("wikipedia.txt", 'a', encoding='utf-8') as file:
      for link, name in links_and_names:
          file.write(f"{link} | {name}\n")
  offset+=500
  print(offset)
  time.sleep(1)

1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000


In [ ]:
import pandas as pd

# Load the Excel file
file_path = 'merged_imdb_mexico.xlsx'  # Replace with your file path
sheet_name = 'Sheet1'  # Replace with the actual sheet name if necessary

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Count non-empty rows in the "Death Date" column
non_empty_count = df['Death Date'].dropna().shape[0]

print(f"Number of non-empty 'Death Date' rows: {non_empty_count}")


Number of non-empty 'Death Date' rows: 2439


In [ ]:
import pandas as pd

# Load the Excel file
file_path = 'merged_imdb_mexico.xlsx'  # Replace with your file path
sheet_name = 'Sheet1'  # Replace with the actual sheet name if necessary

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Filter rows with non-empty "Death Date"
non_empty_df = df[df['Death Date'].notna()]

# Filter rows with empty "Death Date"
empty_df = df[df['Death Date'].isna()]

# Save the non-empty rows to a new Excel file
non_empty_file_path = 'non_empty_death_date.xlsx'
non_empty_df.to_excel(non_empty_file_path, index=False)

# Save the empty rows to a new Excel file
empty_file_path = 'empty_death_date.xlsx'
empty_df.to_excel(empty_file_path, index=False)

print(f"Non-empty 'Death Date' rows saved to: {non_empty_file_path}")
print(f"Empty 'Death Date' rows saved to: {empty_file_path}")

Non-empty 'Death Date' rows saved to: non_empty_death_date.xlsx
Empty 'Death Date' rows saved to: empty_death_date.xlsx
